In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import time
import json
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import unquote_plus, quote_plus
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv("./data/save-scrap-meteo.csv", sep=';')

In [5]:
df

,date,temp,wind_dir,wind_speed,hum,press,weather_event
0,2019-01-01 23:00,8.0,NNE,11.1,79,1036.6(1.1/3h),--
1,2019-01-01 22:00,8.6,NNO,3.7,90,1036(0.6/3h),--
2,2019-01-01 21:00,8.6,N,5.6,90,1035.7(0.3/3h),--
3,2019-01-01 20:00,8.5,NNO,5.6,90,1035.5(0.3/3h),--
4,2019-01-01 19:00,8.3,NO,7.4,90,1035.4(0.8/3h),--
...,...,...,...,...,...,...,...
26010,2021-12-31 04:00,9.0,S,5.6,96,1021.5(-0.5/3h),--
26011,2021-12-31 03:00,9.5,S,11.1,96,1021.4(-0.8/3h),Ciel clair
26012,2021-12-31 02:00,9.6,SO,5.6,96,1022(-0.9/3h),Ciel clair
26013,2021-12-31 01:00,10.3,S,7.4,96,1022(-1.1/3h),Ciel clair


In [7]:
startdate = datetime.strptime("2019-01-01 00:00", "%Y-%m-%d %H:%M")
enddate = datetime.strptime("2021-12-11 00:00", "%Y-%m-%d %H:%M")
days = [datetime.strftime(startdate + timedelta(hours=i), "%Y-%m-%d %H:%M") for i in range((enddate - startdate).days*24 + 1)]
df_days = pd.DataFrame(days, columns=["date"])

In [17]:
merge = pd.merge(df_days, df, on="date", how="outer").drop_duplicates()
merge

,date,temp,wind_dir,wind_speed,hum,press,weather_event
0,2019-01-01 00:00,8.6,ONO,5.6,83,1036.1(-0.4/3h),--
1,2019-01-01 01:00,8.2,ONO,3.7,84,1036(-0.4/3h),--
2,2019-01-01 02:00,7.9,NNO,3.7,90,1036.2(-0.1/3h),--
3,2019-01-01 03:00,7.7,O,3.7,88,1035.8(-0.3/3h),--
4,2019-01-01 04:00,7.9,ONO,3.7,82,1035.4(-0.6/3h),--
...,...,...,...,...,...,...,...
26545,2021-12-31 04:00,9.0,S,5.6,96,1021.5(-0.5/3h),--
26546,2021-12-31 03:00,9.5,S,11.1,96,1021.4(-0.8/3h),Ciel clair
26547,2021-12-31 02:00,9.6,SO,5.6,96,1022(-0.9/3h),Ciel clair
26548,2021-12-31 01:00,10.3,S,7.4,96,1022(-1.1/3h),Ciel clair


In [18]:
# display rows with nan
merge[merge.isna().any(axis=1)]

,date,temp,wind_dir,wind_speed,hum,press,weather_event
30,2019-01-02 06:00,NaN,NaN,NaN,NaN,NaN,NaN
150,2019-01-07 06:00,NaN,NaN,NaN,NaN,NaN,NaN
342,2019-01-15 06:00,NaN,NaN,NaN,NaN,NaN,NaN
408,2019-01-18 00:00,NaN,NaN,NaN,NaN,NaN,NaN
409,2019-01-18 01:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
25582,2021-11-21 06:00,NaN,NaN,NaN,NaN,NaN,NaN
25678,2021-11-25 06:00,NaN,NaN,NaN,NaN,NaN,NaN
25798,2021-11-30 06:00,NaN,NaN,NaN,NaN,NaN,NaN
25870,2021-12-03 06:00,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df[df.duplicated()]

,date,temp,wind_dir,wind_speed,hum,press,weather_event
417,2019-01-18 12:00,4.9,S,5.6,68,1020.3(-0.4/3h),--
420,2019-01-18 10:00,2.5,S,3.7,80,1020.9(0.3/3h),--
422,2019-01-18 09:00,1.9,S,1.9,84,1020.8(0.3/3h),--
427,2019-01-18 04:00,0.8,S,3.7,85,1020.3(0.0/3h),--
443,2019-01-19 10:00,0.3,SSE,7.4,82,1013.3(-0.2/3h),--
...,...,...,...,...,...,...,...
11935,2020-05-16 12:00,17.3,N,3.7,36,1023.4(-0.6/3h),--
11937,2020-05-16 11:00,18.1,E,3.7,39,1023.5(-0.6/3h),--
19375,2021-03-28 03:00,7.2,SO,5.6,70,1029.1(-0.6/3h),--
19376,2021-03-28 03:00,7.2,SO,5.6,70,1029.1(-0.6/3h),--
